In [63]:
import pandas as pd
import os
import datetime
import numpy as np

In [55]:
def getMeanXYZ(df):
    if df.empty:
        return 
    #removing excess data so that data falls in 1 second data intervals
    maxIndx = int(len(df)/100)*100
    df = df.iloc[0:maxIndx]
    meanPerSecondDf = pd.DataFrame()
    
    for i in range(0,int(maxIndx/100)):
        startIndx = i*100
        endIndx = startIndx+100
        tempdf = df.iloc[startIndx:endIndx]
        
        meanDf = tempdf.mean(axis=0)
        stdDf = tempdf.std()
        
        meanPerSecondDf = meanPerSecondDf.append({'mean_x':meanDf.X, 'std_x':stdDf.X, 'mean_y':meanDf.Y,'std_y':stdDf.Y, 'mean_z':meanDf.Z, 'std_z':stdDf.Z},ignore_index=True)
        
    return meanPerSecondDf


In [56]:
def getMeanXYZ_updated(df):
    startInd = 0
    endInd = 1

    meanPerSecondDf = pd.DataFrame()
    updateFlag = False
    for i in range(0, len(df)):
        if(df.iloc[i].time - df.iloc[startInd].time > 60):
            updateFlag = True

        if updateFlag==True:
            updateFlag = False
            tempdf = df.iloc[startInd:i-1]
            startInd = i

            meanDf = tempdf.mean(axis=0)
            stdDf = tempdf.std()
            meanPerSecondDf = meanPerSecondDf.append({'mean_x':meanDf.X, 'std_x':stdDf.X, 'mean_y':meanDf.Y,'std_y':stdDf.Y, 'mean_z':meanDf.Z, 'std_z':stdDf.Z},ignore_index=True)
    return meanPerSecondDf

In [57]:
#trying out stuff

#need to do this for all files
#reading the file with no header
df = pd.read_csv("Data/G7NZCJ008056297-Justin-left-hand_wash--1970-01-01-00-59-42.csv", header=None)
#renaming the columns
df = df.rename({0:'time', 1: 'X', 2: 'Y', 3: 'Z'},axis='columns')
meanDf = getMeanXYZ(df)
meanDf.iloc[1].mean_x

-1.4533337856999997

In [58]:
# temp = pd.to_datetime(df['time'].astype('str'), format ="%d/%m %H:%M:%S")
temp = df.time
tempDf = pd.DataFrame()

startInd = 0
endInd = 1

meanPerSecondDf = pd.DataFrame()

updateFlag = False
for i in range(0, len(df)):
    if(df.iloc[i].time - df.iloc[startInd].time > 60):
        updateFlag = True
    
    if updateFlag==True:
        updateFlag = False
        tempdf = df.iloc[startInd:i-1]
        startInd = i
        
        meanDf = tempdf.mean(axis=0)
        stdDf = tempdf.std()
        meanPerSecondDf = meanPerSecondDf.append({'mean_x':meanDf.X, 'std_x':stdDf.X, 'mean_y':meanDf.Y,'std_y':stdDf.Y, 'mean_z':meanDf.Z, 'std_z':stdDf.Z},ignore_index=True)
meanPerSecondDf

,mean_x,std_x,mean_y,std_y,mean_z,std_z
0,-1.175955,NaN,3.323090,NaN,8.359816,NaN
1,-0.273232,0.090832,2.524550,0.056997,8.912466,0.080559
2,-0.444276,0.130098,2.394421,0.149198,8.747410,0.497712
3,-1.386716,0.499137,1.396496,1.386063,6.473738,0.529306
4,-0.326920,0.273794,-4.757706,1.973065,3.909470,0.187076
...,...,...,...,...,...,...
111,-2.874023,0.129025,1.831791,0.313955,7.971623,0.094056
112,-2.581352,0.086911,1.876737,0.220258,8.058283,0.130200
113,-2.638912,0.131390,2.037164,0.346737,8.420490,0.112269
114,-2.155518,0.932770,1.760140,0.965102,8.746013,0.809792


In [59]:
#main code
dataPath = 'Data/'
handWashSubString = 'left-hand_wash'
notHandWashSubstring = 'left-not_hand_wash'

handWashedDf = pd.DataFrame()
notHandWashedDF = pd.DataFrame()

#Reading through the data folder for filenames
for path in os.listdir(dataPath):
    filepath = dataPath+path
    df = pd.read_csv(filepath, header=None)
    df = df.rename({0:'time', 1: 'X', 2: 'Y', 3: 'Z'},axis='columns')
    meanDf = getMeanXYZ_updated(df)
        
    if handWashSubString in path:
        handWashedDf = handWashedDf.append(meanDf,ignore_index=True)
    else:
        notHandWashedDF = notHandWashedDF.append(meanDf,ignore_index=True)

In [60]:
handWashedDf['Activity'] = 'hand_wash'
handWashedDf

,mean_x,std_x,mean_y,std_y,mean_z,std_z,Activity
0,-1.175955,NaN,3.323090,NaN,8.359816,NaN,hand_wash
1,-0.273232,0.090832,2.524550,0.056997,8.912466,0.080559,hand_wash
2,-0.444276,0.130098,2.394421,0.149198,8.747410,0.497712,hand_wash
3,-1.386716,0.499137,1.396496,1.386063,6.473738,0.529306,hand_wash
4,-0.326920,0.273794,-4.757706,1.973065,3.909470,0.187076,hand_wash
...,...,...,...,...,...,...,...
2135,-1.664738,0.321739,0.684976,0.410565,8.677755,0.299367,hand_wash
2136,-1.718826,0.241279,1.071172,0.160818,8.923843,0.147916,hand_wash
2137,-1.626019,0.285427,0.535087,0.165410,9.031020,0.206379,hand_wash
2138,-1.686293,0.062731,1.145618,0.121025,8.474976,0.910035,hand_wash


In [61]:
notHandWashedDF['Activity'] = 'no_hand_wash'
notHandWashedDF

,mean_x,std_x,mean_y,std_y,mean_z,std_z,Activity
0,-0.714315,0.012702,3.654201,0.021169,8.735235,0.144797,no_hand_wash
1,-0.379810,0.081871,3.152245,0.180025,8.087183,0.102505,no_hand_wash
2,-0.086221,0.178605,2.447111,0.232073,7.615364,0.206627,no_hand_wash
3,0.469623,0.217903,1.969105,0.427593,6.936575,0.431405,no_hand_wash
4,-0.085023,0.134018,1.301094,0.436409,9.602232,1.522108,no_hand_wash
...,...,...,...,...,...,...,...
5210,0.549258,0.689967,3.537643,0.452495,9.006072,0.822515,no_hand_wash
5211,-0.285007,0.286155,3.859175,0.352427,8.902088,0.421633,no_hand_wash
5212,-0.962199,0.596523,4.897416,0.413638,6.912226,0.264319,no_hand_wash
5213,-1.440844,4.697989,3.935975,0.600644,10.015493,5.717826,no_hand_wash


In [64]:
nameOfTheFile = 'processedDataAssignment1.csv'
to_save_df = pd.DataFrame()
to_save_df = to_save_df.append(handWashedDf,ignore_index=True)
to_save_df = to_save_df.append(notHandWashedDF,ignore_index=True)

to_save_df['std_x'].replace('', np.nan, inplace=True)
to_save_df.dropna(subset=['std_x'], inplace=True)

to_save_df.to_csv(nameOfTheFile, index=False,encoding='utf-8')